In [1]:
import pandas as pd
import networkx as nx
import ast
import json

data = pd.read_csv('data/sample-summary.csv')
relation = pd.read_csv('data/sample-relation.csv')

In [2]:
doi_list = data['doi'].to_list()

def get_name(doi):
    return data.loc[data['doi'] == doi].title.values[0]

In [3]:
G = nx.DiGraph()
for i in range(len(data)):
    paper = data.iloc[i]
    summaries = ast.literal_eval(paper.summaries)
    for i, summary in enumerate(summaries):
        G.add_node(f"{paper.doi}:{i}", name=summary['summary'], effect=summary['effect'], paper=paper['doi'], kind="finding")
    G.add_node(f"{paper['doi']}", name=get_name(paper['doi']), kind="paper", summaries=len(summaries), theme=paper['theme'], keywords=paper['keywords'], year=paper['year'].item())

# add literal references
for i in range(len(data)):
    paper = data.iloc[i]
    references = ast.literal_eval(paper.references)
    for ref in references:
        if(ref in doi_list):
            # (f"{paper.doi}", f"{ref}", kind="paper")
            G.add_edge(f"{paper.doi}", f"{ref}", kind="paper")
            
# add paper to finding            
for i in range(len(data)):
    paper = data.iloc[i]
    summaries = ast.literal_eval(paper.summaries)
    for i, summary in enumerate(summaries):
        G.add_edge(f"{paper.doi}", f"{paper.doi}:{i}", kind="paper_to_finding")

In [4]:
# add relation
for i in range(len(relation)):
    # paper = relation.iloc[i]
    # G.add_edge(f"{paper['paper']}", f"{paper['finding']}", kind="paper_to_finding")
    # print(paper['paper'], paper['finding'])
    r = relation.iloc[i]
    source = r.iloc[1]
    target = r.iloc[2]
    p = ast.literal_eval(r.iloc[3])
    G.add_edge(f"{source}", f"{target}", relation=p['relation'], explanation=p['explanation'], polarity=p['polarity'], kind="finding")

In [5]:
graph_data = nx.node_link_data(G, edges="edges")
with open('graph-relation.json', 'w') as f:
    f.write(json.dumps(graph_data))